## So far:
pad sentence and record ending index

But cannot index properly...

So how can I do batching??? Well, pad it with . zero. Just let it learn? How the effect of 0 will not be too big

In [2]:
from data_pre import data_preprocessing
from data_pre import data_preprocessing
from WAC import WAC
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
import time
from torch.utils.data import DataLoader, TensorDataset

(voc_ix, train_data,test_data, dev_data) = data_preprocessing()


runtime: 1.5681426525115967


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
from WAC_ATT import WAC_ATT


'''
load and prepare data
'''
#(voc_ix, trainX, trainy, testX, testy, devX, devy) = data_preprocessing()
#print("finish preparing data\n")

'''
set parameters
'''
## set hyperparameters
VOCAB_SIZE = len(voc_ix) + 1
EMBEDDING_DIM = 100
n_epoch = 20
batch_size = 500
eval_per = 20000/batch_size
PATH = "../model/wac_batch.pt"

## define model
model = WAC_ATT(EMBEDDING_DIM, VOCAB_SIZE)
optimizer = optim.Adagrad(model.parameters(), lr = 1e-2)

## Experiment with training

In [25]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
dataiter = iter(train_loader)
X,y,lens = dataiter.next()

In [26]:
X.shape

torch.Size([500, 56])

In [27]:
VOCAB_SIZE = len(voc_ix) + 1
embedding_dim = 100

In [28]:
word_embeddings = nn.Embedding(VOCAB_SIZE, embedding_dim, sparse = True)
u = nn.Embedding(1,embedding_dim) 

In [29]:
embeds = word_embeddings(X)
embeds.shape

torch.Size([500, 56, 100])

In [30]:
cosine = nn.CosineSimilarity(dim = 2)

In [31]:
maxlen = X.size(1)
mask = torch.arange(maxlen)[None,:] < lens[:,None]

## compute attention
att = torch.exp(cosine(u.weight.data.view(1,1,-1), embeds))

att[~mask] = float(0) 
att = att/att.sum()

In [34]:
bs, maxlen = X.size()

In [39]:
att.size()

torch.Size([500, 56])

In [38]:
X.size()

torch.Size([500, 56])

In [37]:
mask.size()

torch.Size([500, 56])

In [41]:
torch.mul(mask.float(), att)

tensor([[0.0002, 0.0002, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0002, 0.0002, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
        [0.0002, 0.0002, 0.0002,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)

In [21]:
att.sum(dim = 1, keepdim=True).shape

torch.Size([500, 1])

In [23]:
att /= att.sum(dim = 1, keepdim=True)

In [24]:
att.shape

torch.Size([500, 56])

In [20]:
(att.t()/att.sum(dim = 1)).t()

tensor([[0.0739, 0.0818, 0.0718,  ..., 0.0000, 0.0000, 0.0000],
        [0.1399, 0.1494, 0.1275,  ..., 0.0000, 0.0000, 0.0000],
        [0.0874, 0.0821, 0.0744,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.3660, 0.3153, 0.3187,  ..., 0.0000, 0.0000, 0.0000],
        [0.0283, 0.0260, 0.0356,  ..., 0.0000, 0.0000, 0.0000],
        [0.1749, 0.1519, 0.1619,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<TBackward>)

In [13]:
torch.mul(att.view(att.size()[0],att.size()[1],1), embeds).shape

torch.Size([500, 56, 100])

In [69]:
torch.sum?

In [68]:
att.view(1,att.size()).shape

TypeError: view(): argument 'size' must be tuple of ints, but found element of type torch.Size at pos 2

In [45]:
att.shape

torch.Size([500, 56])

In [41]:
embeds.shape

torch.Size([500, 56, 100])

In [46]:
torch.matmul?

In [44]:
torch.mul(embeds, embeds).sum(dim = 2).shape

torch.Size([500, 56])

In [53]:
lens

tensor([ 2, 15, 27,  3,  6,  1,  2, 33, 15, 10,  3,  3, 10, 25, 12,  1, 10,  1,
         3,  6,  7, 13,  3, 11,  2,  2,  3, 25, 18,  7,  6,  2, 18, 12, 16, 10,
         5,  2,  4, 17,  4, 25,  5, 19,  8,  1, 13,  4, 14,  1, 34, 17, 13,  8,
        11, 10,  7, 11,  4, 17,  4,  2, 27, 25,  1, 20,  2, 12,  2, 19, 28, 24,
         1, 16,  3,  1, 12, 12, 10,  5, 31,  8,  8,  6, 22,  7,  3, 21,  3,  5,
         1,  8,  3,  9,  5,  5, 12,  6,  3,  8,  2,  2,  3, 17,  1,  8,  2,  3,
        16, 28,  6,  5,  2, 23,  2,  1,  4,  8, 23,  9,  4, 24,  4,  1,  2, 16,
         9, 16, 31,  2,  1,  5, 24,  2,  7, 10, 12,  2,  2,  3, 11,  3,  9,  5,
         5, 32, 30, 11, 20,  3, 23, 20,  4,  9, 16, 17,  4,  1,  3,  2,  7,  2,
         2, 39,  2, 11, 15, 14,  4, 25, 21, 15, 14,  9,  6,  2,  6, 20,  3, 12,
        17,  2, 12,  1, 36,  8, 15,  1, 15, 12,  9,  4,  2,  4, 10,  4,  2,  7,
         8,  9,  6,  2, 25, 12,  2,  2, 27, 23, 22,  7,  5,  7,  8,  8,  2,  5,
         4, 15, 15, 12,  7,  5,  1,  4, 

In [48]:
mask.shape

torch.Size([500, 56])

In [49]:
embeds.shape

torch.Size([500, 56, 100])

In [73]:
embeds_mean = torch.mul(mask.unsqueeze(2).float(), embeds).mean(dim = 1)
embeds_mean = embeds_mean.mul((maxlen/lens).unsqueeze(1).float())

In [65]:
embeds_mean[0,:]

tensor([ 1.2545e-02, -9.0677e-03, -3.6350e-02,  1.9701e-02,  3.3840e-02,
         4.4991e-02, -2.3492e-03,  3.3255e-02,  2.9349e-02, -1.3542e-02,
        -2.9385e-02,  2.1722e-02,  5.1253e-03, -8.0544e-03, -1.4661e-02,
         2.0961e-03,  3.4882e-02,  4.9197e-02, -1.5508e-02, -3.2877e-02,
         1.4496e-02, -3.3729e-03,  5.5681e-02,  2.1760e-02,  2.9044e-02,
        -3.2889e-02,  1.4972e-03,  1.8613e-02,  4.5386e-02,  2.1909e-02,
        -1.1360e-02, -2.3841e-02, -9.9702e-03, -1.2315e-02,  1.7632e-03,
         2.7768e-02,  6.8033e-03, -4.7548e-03,  1.7184e-02,  1.1790e-03,
         1.3299e-02, -4.1601e-03, -1.4265e-02, -7.1131e-03, -3.5834e-02,
        -4.2925e-02,  3.7214e-02,  5.4426e-03,  3.5782e-02, -3.3906e-03,
         2.8505e-02,  1.5165e-02, -1.3841e-02, -2.0364e-02,  2.6180e-02,
        -2.7734e-02,  7.9024e-03,  3.9405e-02,  4.1891e-02, -4.0047e-02,
        -1.2652e-02,  2.7053e-02, -3.0833e-02,  1.3080e-02, -2.0985e-02,
         1.2611e-02,  4.4108e-02,  7.1553e-03,  9.1

In [67]:
embeds_mean2 = embeds_mean.mul((maxlen/lens).unsqueeze(1).float())

In [69]:
(embeds_mean2/embeds_mean).shape

torch.Size([500, 100])

In [72]:
(embeds_mean2/embeds_mean)[1,:]

tensor([3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000], grad_fn=<SliceBackward>)

In [71]:
maxlen/lens

tensor([28,  3,  2, 18,  9, 56, 28,  1,  3,  5, 18, 18,  5,  2,  4, 56,  5, 56,
        18,  9,  8,  4, 18,  5, 28, 28, 18,  2,  3,  8,  9, 28,  3,  4,  3,  5,
        11, 28, 14,  3, 14,  2, 11,  2,  7, 56,  4, 14,  4, 56,  1,  3,  4,  7,
         5,  5,  8,  5, 14,  3, 14, 28,  2,  2, 56,  2, 28,  4, 28,  2,  2,  2,
        56,  3, 18, 56,  4,  4,  5, 11,  1,  7,  7,  9,  2,  8, 18,  2, 18, 11,
        56,  7, 18,  6, 11, 11,  4,  9, 18,  7, 28, 28, 18,  3, 56,  7, 28, 18,
         3,  2,  9, 11, 28,  2, 28, 56, 14,  7,  2,  6, 14,  2, 14, 56, 28,  3,
         6,  3,  1, 28, 56, 11,  2, 28,  8,  5,  4, 28, 28, 18,  5, 18,  6, 11,
        11,  1,  1,  5,  2, 18,  2,  2, 14,  6,  3,  3, 14, 56, 18, 28,  8, 28,
        28,  1, 28,  5,  3,  4, 14,  2,  2,  3,  4,  6,  9, 28,  9,  2, 18,  4,
         3, 28,  4, 56,  1,  7,  3, 56,  3,  4,  6, 14, 28, 14,  5, 14, 28,  8,
         7,  6,  9, 28,  2,  4, 28, 28,  2,  2,  2,  8, 11,  8,  7,  7, 28, 11,
        14,  3,  3,  4,  8, 11, 56, 14, 

In [64]:
# obtain one batch of training data
batch_size = 500
train_loader = DataLoader(train_data, shuffle=F, batch_size=batch_size)

dataiter = iter(train_loader)
X, y,lens = dataiter.next()

embeds = model.word_embeddings(X)
embeds.shape

torch.Size([500, 56, 100])

In [65]:
X.shape

torch.Size([500, 56])

In [72]:
maxlen = X.size(1)
mask = torch.arange(maxlen)[None,:] < lens[:,None]
embeds[~mask] = float(0)

In [75]:
X[0,:]

tensor([12407, 18639,  9777, 19227, 11647, 10646,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0])

In [76]:
(embeds.sum(dim = 1).t()/lens.float()).t()[0,:]/model.word_embeddings(X[0,:6]).mean(dim = 0)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], grad_fn=<DivBackward0>)

In [62]:
model.word_embeddings(X[0,:9]).shape

torch.Size([9, 100])

In [71]:
lens

tensor([ 6,  1,  4,  3, 10,  9,  3, 20,  8, 21, 10, 12,  7, 13,  6, 13,  9, 12,
         6, 13,  5,  2,  6,  3,  2, 11, 20, 15, 25,  3, 13,  4,  6, 10,  2,  3,
         3, 27,  5, 19,  9, 18,  3,  5,  1,  9, 19, 15,  6, 14,  4,  6, 11, 18,
         9,  6, 14,  8,  5, 22, 10, 15, 10,  4,  8, 13,  4, 17,  7,  6,  1, 23,
         9, 27,  2,  7,  5,  1,  5,  4, 12,  2, 27, 23, 31,  3, 25,  8,  7,  4,
         6,  5, 19,  8,  4,  1,  6, 14,  5,  2,  3, 15,  4,  9, 11,  1,  2,  9,
         1,  8, 30, 14,  3, 15, 15,  2,  4, 16, 11,  4,  4,  2, 13, 14, 12,  8,
        12,  2,  2, 19, 12,  6, 10, 19,  6, 16,  7,  3,  7, 34, 10,  6,  8,  2,
         4,  5,  1, 18, 16,  5, 18, 10,  8,  6, 22,  1,  7,  2,  2,  8,  2,  2,
         7,  1,  6,  6,  4, 12, 33, 17,  7,  2,  1, 20,  8,  5,  1,  8,  2,  5,
         5,  1, 34,  3, 11,  1,  7,  3, 10,  2,  5, 30,  8,  9,  2, 25, 11, 18,
         4, 11,  4,  3,  3,  9,  8, 10,  6,  3,  3,  2,  4,  8,  3, 33, 15,  3,
         3,  7, 13, 10,  9,  3, 28,  3, 

In [56]:
maxlen/lens

tensor([ 6,  8, 14,  9, 28,  9,  2,  9,  5, 18, 28,  2, 56, 14,  6, 56,  9,  3,
         4, 56,  4,  2, 56, 11, 56,  2,  3,  8,  5,  7, 11,  4,  4,  4,  8,  5,
         8,  8, 18,  8,  8,  5, 28,  3, 11,  4,  4,  4, 28, 14,  4, 18, 14, 28,
         9, 18,  2,  2,  1,  9,  3, 28, 18, 56, 11,  5, 56,  7, 14,  8,  6, 56,
        56, 28,  4,  7,  2, 14, 14, 28,  4,  4, 18,  2, 56,  5, 28,  3,  2,  9,
         4, 28,  2,  3,  3,  3,  2,  2,  9, 11,  4, 14, 18, 11, 28, 14,  4, 18,
         2, 14,  3, 18, 18, 28, 11, 56,  5,  4,  5,  4,  2, 14,  9, 28, 56,  2,
        14,  8,  7,  7, 14, 28,  4,  9,  2, 14, 56,  5,  7,  9, 14,  2, 18,  4,
        56,  2,  5, 11, 18,  4, 11,  4,  5,  3,  9, 11,  9,  7,  3,  5, 28,  2,
        56, 56,  5, 28,  8,  5, 56,  1, 56, 56, 18, 18,  3,  4, 28,  4,  7,  3,
         8,  9,  4, 28,  6,  8,  4,  6,  2, 28, 56, 18,  1,  7,  8, 56,  3,  2,
         6,  6,  6, 56, 18, 56, 28,  2,  7,  6, 11,  7, 28, 14,  7,  6, 56, 14,
         2, 28,  6,  4,  3,  2,  9,  1, 

In [22]:
embeds.shape

torch.Size([500, 56, 100])

In [27]:
embeds[2,6,:]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], grad_fn=<SliceBackward>)

In [44]:
(embeds.mean(dim = 1).t()*(maxlen/lens.float())).t()[0,:].shape

torch.Size([100])

In [38]:
torch.matmul(maxlen/lens.float(),embeds.mean(dim = 1)).shape

torch.Size([100])

In [5]:
model.predict(X)

RuntimeError: t() expects a 2D tensor, but self is 3D

In [6]:
model.evaluate(dev_data.tensors)

0.5355504587155964

In [16]:
dev_data.tensors[1].shape

torch.Size([872])

In [27]:
x = dev_data.tensors[0][0,:]
y = dev_data.tensors[1][0]

model.forward(x.unsqueeze(0),lens = torch.Tensor(1))

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'other'

In [35]:
model.forward(x.unsqueeze(0),lens = torch.tensor([1], dtype = torch.long))

tensor([[0.4732]], grad_fn=<SigmoidBackward>)

In [ ]:
dtype=torch.long, device=device)